In [1]:
from routrie import Router as RoutrieRouter
from http_router import Router as HTTPRouter
from starlette.routing import Route, Router as StarletteRouter
from starlette.requests import Request

In [ ]:
routrie_router: RoutrieRouter[int] = RoutrieRouter()
routrie_router.insert("/path", 1)
routrie_router.insert("/path/users", 2)

In [ ]:
%%timeit
for _ in range(10_000):
    routrie_router.find("/path")